In [ ]:
import json, time
from IPython.display import clear_output
from STservo import ST3215, Calibrator 

class MAPPER:
    def __init__(self):
        with open('config/Ctrl.json', 'r') as fr: 
            conf_ctrl = json.load(fp=fr)
        with open('config/Arm.json'       , 'r') as fr:
            conf_arm = json.load(fp=fr)
        
        self.dev_id = list(range(1, 13))
        for id_int in self.dev_id:
            id_str = str(id_int)
            if conf_ctrl[id_str]['min'] > 2047:
                conf_ctrl[id_str]['min'] = conf_ctrl[id_str]['min'] - conf_ctrl[id_str]['corr'] - 4096
            else:
                conf_ctrl[id_str]['min'] = conf_ctrl[id_str]['min'] - conf_ctrl[id_str]['corr']
            if conf_arm [id_str]['min'] > 2047:
                conf_arm[ id_str]['min'] = conf_arm [id_str]['min'] - conf_arm [id_str]['corr'] - 4096
            else:
                conf_arm[ id_str]['min'] = conf_arm [id_str]['min'] - conf_arm [id_str]['corr']

        self.conf_ctrl = conf_ctrl
        self.conf_arm  = conf_arm 

    def getPosiVelo(self, result):
        arm_dict = {}
        for id_int in self.dev_id:
            id_str = str(id_int)
            ratio  = (result[id_int]['posi'] - self.conf_ctrl[id_str]['min']) / self.conf_ctrl[id_str]['range']
            posi   = self.conf_arm[id_str]['min'] + self.conf_arm[id_str]['range'] * ratio
            arm_dict[id_str] = {'posi': int(posi), 'velo': 1200}
        arm_json = json.dumps(arm_dict)
        return len(arm_json).to_bytes(length=2, byteorder='big') + arm_json.encode()
m = MAPPER()
m.conf_arm, m.conf_ctrl

In [ ]:
import socket

servo_ctrl = ST3215(port='/dev/ttyACM0')
for _id in m.dev_id:
    servo_ctrl.__set_torque_mode__(dev_id=_id, mode='free')
    
HOST = '192.168.3.123'   # 服务器地址
PORT = 5000              # 服务器端口
try:
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    client.connect((HOST, PORT))
    while True:
        result = servo_ctrl.readPosi(id_arr=m.dev_id, debug=True)
        client.sendall(m.getPosiVelo(result))
        assert b'\xff\xff' == client.recv(2)
            
except KeyboardInterrupt:
    client.sendall(b'\x00\x00')
    pass 